In [80]:
from skimage.io import imread
image = imread('parrots.jpg')

In [81]:
from skimage import img_as_float

float_image = img_as_float(image)
X_list = []
for row in float_image:
    for elem in row:
        X_list.append(elem)

In [82]:
from pandas import DataFrame

X = DataFrame(X_list, columns=['red','green','blue'])


In [176]:
from sklearn.cluster import KMeans

n_clusters = 11
rofl = KMeans(n_clusters=n_clusters, init='k-means++', random_state=241)
rofl.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=11, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=241, tol=0.0001, verbose=0)

In [157]:
im_high, im_len = float_image.shape[:2]

In [177]:
from statistics import mean, median

clusters = rofl.labels_
arr = []

for i in range(0, n_clusters):
    arr.append([])

for row_num, row in enumerate(float_image):
    for elem_num, elem in enumerate(row):
        clust_type = clusters[ row_num*im_len + elem_num ] # 1:n_clusters
        arr[clust_type - 1].append(float_image[row_num, elem_num])

In [178]:
import numpy as np

clust_means = []
clust_medians = []

for i in range(1,n_clusters+1):
    color_list = np.matrix(arr[i-1]) # [ [r,g,b], [r,g,b],...]

    tmp_tuple_r = list(map(lambda x: x[0], np.array(color_list[:,0])))
    tmp_tuple_g = list(map(lambda x: x[0], np.array(color_list[:,1])))
    tmp_tuple_b = list(map(lambda x: x[0], np.array(color_list[:,2])))

    mean_r = mean( tmp_tuple_r )
    mean_g = mean( tmp_tuple_g )
    mean_b = mean( tmp_tuple_b )
    
    median_r = median( tmp_tuple_r )
    median_g = median( tmp_tuple_g )
    median_b = median( tmp_tuple_b )
    
    clust_means.append( list( (mean_r, mean_g, mean_b) ) )
    clust_medians.append( list( (median_r, median_g, median_b) ) )

In [179]:
mean_image = float_image.copy()

for row_num, row in enumerate(float_image):
    for elem_num, elem in enumerate(row):
        clust_type = clusters[ row_num*im_len + elem_num ] # 1:n_clusters
        mean_image[row_num, elem_num] = clust_means[clust_type - 1]

In [161]:
from skimage.viewer import ImageViewer

viewer = ImageViewer(mean_image)
viewer.show()

[]

In [162]:
from skimage.io import imsave

imsave('mean_parrots.png', mean_image)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [180]:
median_image = float_image.copy()

for row_num, row in enumerate(float_image):
    for elem_num, elem in enumerate(row):
        clust_type = clusters[ row_num*im_len + elem_num ] # 1:n_clusters
        median_image[row_num, elem_num] = clust_medians[clust_type - 1]

In [164]:
imsave('median_parrots.png', median_image)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [182]:
from math import log10

def psnr(I, K, I_len, I_high):
    MSE = 0
    for i in range(0, I_high):
        for j in range(0, I_len):
            irgb = np.array(I[i,j])
            krgb = np.array(K[i,j])
            delta = irgb - krgb
            MSE += np.dot(delta, delta)
    MSE /= I_len*I_high*3
    
    return 20*log10(1) - 10*log10(MSE)

In [183]:
psnr(float_image, mean_image, im_len, im_high) # mean -> 13.67 (n_clusters == 8)

20.127588746515897

In [184]:
psnr(float_image, median_image, im_len, im_high) # median -> 13.37 (n_clusters == 8)

19.841284073901345